In [3]:
from tools import *

In [4]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Load dataset (assuming train_raw and test_raw are available)
train_raw = train_raw.copy()
test_raw = test_raw.copy()

# Select features and target
features = [f"x{i}" for i in range(1, 14)]
X_train = train_raw[features]
y_train = train_raw['y']
X_test = test_raw[features]  # 10,000 test data points for Kaggle submission

# Encode categorical target variable
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)  # Converts ['Anthropic', 'OpenAI', 'Mistral'] to [0, 1, 2]

# Convert boolean columns to integers (if any)
bool_cols = X_train.select_dtypes(include=['bool']).columns
X_train[bool_cols] = X_train[bool_cols].astype(int)
X_test[bool_cols] = X_test[bool_cols].astype(int)

# Encode categorical features (if any)
cat_cols = X_train.select_dtypes(include=['object']).columns
for col in cat_cols:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])  # Apply the same encoding to test

# XGBoost parameters
params = {
    "objective": "multi:softmax",  # Multiclass classification
    "num_class": len(np.unique(y_train)),  # Number of classes
    "eval_metric": "mlogloss",  # Multi-class log loss (good for Kaggle)
    "learning_rate": 0.1,
    "max_depth": 6,
    "n_estimators": 300,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "random_state": 42
}

# K-Fold Cross Validation (Stratified)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = []
for train_idx, val_idx in kf.split(X_train, y_train):
    X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

    # Train XGBoost
    model = xgb.XGBClassifier(**params)
    model.fit(X_train_fold, y_train_fold)

    # Validate model
    y_val_pred = model.predict(X_val_fold)
    acc = accuracy_score(y_val_fold, y_val_pred)
    cv_scores.append(acc)

print(f"Cross-validation accuracy: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}")

# Train final model on full training data
final_model = xgb.XGBClassifier(**params)
final_model.fit(X_train, y_train)

# Predict on test set
y_test_pred = final_model.predict(X_test)

# Convert predictions back to original labels
y_test_pred_labels = label_encoder.inverse_transform(y_test_pred)

# Save for Kaggle submission
submission = pd.DataFrame({"id": test_raw.index, "y": y_test_pred_labels})
submission.to_csv("submission.csv", index=False)

print("Submission file saved as submission.csv 🚀")


C:\Users\natha\AppData\Local\Temp\ipykernel_12992\783012097.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[bool_cols] = X_train[bool_cols].astype(int)
C:\Users\natha\AppData\Local\Temp\ipykernel_12992\783012097.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[col] = le.fit_transform(X_train[col])


Cross-validation accuracy: 0.6752 ± 0.0157
Submission file saved as submission.csv 🚀
